In [ ]:
import os

In [ ]:
os.chdir(os.path.split(os.getcwd())[0])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [ ]:
import matplotlib.style as style

In [ ]:
style.use('seaborn-poster')

### Parse raw logs and create DataFrame

In [ ]:
def parse_raw_log(log):
    data = []
    lines = log.split('\n')
    for line in lines[:-1]:
        raw = line.split(': ')
        if raw[0].split(', ')[1] == 'LOG':
            entries = raw[1].split(', ')
            datum = []
            for entry in entries:
                value = entry.split(':')[1]
                try:
                    datum.append(float(value))
                except ValueError:
                    datum.append(value)

            data.append(datum)
    
    return data

In [ ]:
experiment_folder = 'Nov7_DefaultArgs_MultiStrike_HighTrCost'
log_path = os.path.join('experiments', experiment_folder, 'training.log')
with open(log_path, 'r') as f:
    log = f.read()
    
data = parse_raw_log(log)

columns = ['episode', 'step', 'K', 'T', 'S', 'c', 'delta', 'n', 'cost', 'action', 'dn', 'kind', 'epsilon', 'pnl', 'reward', 'best_mean_reward', 'loss']
train_df = pd.DataFrame(data, columns = columns)
train_df = train_df.astype({'episode': int, 'step': int, 'action': int})

In [ ]:
train_df[train_df['episode'] == 15000].head()

### Visualize arbitrary episode

In [ ]:
episode = 15000
episode_df = train_df[train_df['episode'] == episode]

fig, ax = plt.subplots(figsize = (14, 10), nrows = 1, ncols = 1, sharex = True)
ax.plot(episode_df['step'], episode_df['n'], color = 'blue', lw = 1.5, label = 'n')
ax.plot(episode_df['step'], episode_df['delta'] * -100, color = 'orange', lw = 1.5, label = 'delta')
ax.scatter(episode_df[episode_df['kind'] == 'random']['step'], episode_df[episode_df['kind'] == 'random']['n'], color = 'red', label = 'random')#, facecolor = 'None')
ax.scatter(episode_df[episode_df['kind'] == 'policy']['step'], episode_df[episode_df['kind'] == 'policy']['n'], color = 'green', label = 'policy')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10))
ax.plot(episode_df['step'], episode_df['pnl'], color = 'green', lw = 1.5, label = 'pnl')
ax.plot(episode_df['step'], episode_df['reward'], color = 'blue', lw = 1.5, label = 'reward')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (12, 8))
groupby_episode = train_df.groupby('episode').mean()
rewards = train_df.groupby('episode').sum()['reward'].values[:-1]
best_mean_reward = groupby_episode['best_mean_reward'].values[:-1]
mean_loss = groupby_episode['loss'].values[:-1]

ax.plot(np.arange(1, len(rewards) + 1), np.clip(rewards, -5000, None), label = 'reward', color = 'blue', lw = 0.1)
ax.plot(np.arange(1, len(best_mean_reward) + 1), np.clip(best_mean_reward, -5000, None), label = 'best_mean_reward', color = 'red', lw = 1.5)
ax.set_ylabel('reward')
ax.set_xlabel('episode')
#fig.savefig('reward_vs_episode.png', dpi = 300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10), nrows = 2, ncols = 1, sharex = True)
groupby_episode = train_df.groupby('episode').mean()
rewards = train_df.groupby('episode').sum()['reward'].values[:-1]
best_mean_reward = groupby_episode['best_mean_reward'].values[:-1]
mean_loss = groupby_episode['loss'].values[:-10000]

ax[0].plot(np.arange(1, len(rewards) + 1), np.clip(rewards, -5000, None), label = 'reward', color = 'blue', lw = 0.2)
ax[0].plot(np.arange(1, len(best_mean_reward) + 1), np.clip(best_mean_reward, -5000, None), label = 'best_mean_reward', color = 'red', lw = 1.5)
ax[0].set_ylabel('reward')

ax[1].plot(np.arange(1, len(mean_loss) + 1), mean_loss, label = 'loss', color = 'blue', lw = 1)
ax[1].set_xlabel('episode')
ax[1].set_ylabel('loss')
plt.show()

### Validation

In [ ]:
#experiment_folder = 'Oct23_clip100_brc10_ne25000_withvalidation'
log_path = os.path.join('experiments', experiment_folder, 'validation.log')
with open(log_path, 'r') as f:
    log = f.read()
    
data = parse_raw_log(log)

columns = ['train_episode', 'val_episode', 'step', 'S', 'c', 'delta', 'n', 'action', 'dn', 'kind', 'epsilon', 'pnl', 'reward', 'best_mean_reward', 'loss']
val_df = pd.DataFrame(data, columns = columns)
val_df = val_df.astype({'train_episode': int, 'val_episode': int, 'step': int, 'action': int})

In [ ]:
val_df.head()

In [ ]:
val_df['train_episode'].unique()

In [ ]:
eps = [i * 1000 for i in range(1, 6, 1)]

In [ ]:
fig, ax = plt.subplots(nrows = val_df['train_episode'].nunique(), ncols = 1, figsize = (14, 100))
x = np.linspace(-25, 25, 1000)
for i, ep in enumerate(val_df['train_episode'].unique()):
    pnls = val_df[val_df['train_episode'] == ep]['pnl'].values
    kernel = gaussian_kde(pnls)
    y = kernel(x)
    ax[i].plot(x, y, lw = 1.5, label = f'Episode: {ep} | Mean: {np.mean(pnls):.3f}, Std: {np.std(pnls):.3f}')
for a in ax:
    a.set_ylim([0, 0.08])
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10))
x = np.linspace(-25, 25, 10000)
for ep in eps:
    pnls = val_df[val_df['train_episode'] == ep]['pnl'].values
    kernel = gaussian_kde(pnls)
    y = kernel(x)
    ax.plot(x, y, lw = 1.5, label = f'{ep} | {np.mean(pnls):.3f}, {np.std(pnls):.3f}')
plt.legend()
plt.show()